* Dans cette partie nous abordons concrètement les fonctionnalité de pandas pour l’analyse de données. 
* Notamment nous aborderons les  opérateurs usuelles (average min , max , quantile ) , les fonction de transformation pivot et groupby , stack/unstack et sur les time series. 

# import   

In [ ]:
import json
import time
import glob, os
import random

import pandas as pd
import numpy as np

from IPython.display import display, HTML
import matplotlib.pyplot as plt

#permet d'afficher des graph dans le notebook
%matplotlib inline 

#permet d'afficher plusieurs sortis par cellule sans utiliser print()
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#permet d'augemnter la largeur des cellules
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:70%! important; }</style>"))

In [ ]:
#optionnel si difficulté pour autocompletion 
%config Completer.use_jedi = False

# Analyse

In [ ]:
df = pd.read_pickle('df')
df.shape
df.head(2)

## Fonction descriptive

Nous avons déjà rencontré dans la section manipulation les fonctions  values_counts et nunique. Ici nous présentons la fonction describe qui donne de nombreuses informations condensées sur le dataframe (min max , count , quantile …)
1. Le résultat de describe est un dataframe qui peut donc être manipulé ainsi par ex nous pouvons faciliter l’affichage en appliquant les fonctions round ou astype ( int ) 
1. Par défaut cette fonction ne prend que les valeurs numériques mais nous pouvons modifier cette sélection avec l’option  include / exclude . On note dans ce cas  l’apparition de nouvelles métriques en index (top , freq …) . 
1. Il est important de noter que les métriques affichées dans describe sont accessibles directement via des fonctions pandas (min max par ex) mais également le quantile 75% correspond à la fonction quantile a 0.75. 


In [ ]:
df.nunique()

In [ ]:
df.Date_mutation.value_counts()

In [ ]:
df.describe().astype(int)

In [ ]:
df.describe(include = 'all')

In [ ]:
df.describe(exclude = [np.number])

In [ ]:
df.Valeur_fonciere.describe()['75%']
df.Valeur_fonciere.quantile(0.75)

## Groupby

Cette fonction, que l’on retrouve littéralement dans le langage SQL , est utiliser pour grouper les données autour d’une fonction d’agrégation. Elle implique plusieurs opérations pour fonctionner selon la logique split-apply-combine. On suppose ici que les formés sont déjà habitué au concept de groupby.
1. Il faut déterminer la clef de groupe ici région ce qui génère un objet groupby, qui peut être réutiliser ensuite.
1. Sur cet objet on applique une ou plusieurs listes de valeurs et une fonction d’agrégation : ici la moyenne, le résultat obtenu est une nouvelle série si le groupby est réalisé sur 1 seule colonne. 
1. La plupart des fonctions pandas sont accessible comme méthode d’agrégation comme par exemple describe. 
1. Dans le cas où on a besoin de réaliser une opération spécifique par colonnes on peut aussi utiliser la commande agg et pour les groupes de donnés la fonction apply ou map.
1. La clef de groupe peut également être un masque sur les données : ex ici on catégorise les surfaces de terrain supérieur ou inferieur a 100 m2. En python la fonction lambda permet d’utiliser des opérateurs sur chaque donnée x.
1. Astuce : dans le cas de plusieurs clefs de group on peut les renseigner soit avec leur nom ou directement en avec une série de valeur , il faut veiller à ce que les tailles des tableaux correspondent


In [ ]:
df.groupby('Region')

In [ ]:
df.groupby('Region')['Valeur_fonciere'].mean()

In [ ]:
df.groupby('Region')['Valeur_fonciere'].mean().astype(int).plot.barh()

In [ ]:
df.groupby('Region')['Valeur_fonciere'].describe().astype(int).head()

In [ ]:
# Surface_terrain  > 100
df.groupby((df.Surface_terrain  > 100))['Valeur_fonciere'].mean().astype(int)

In [ ]:
mask = (df.Surface_terrain  > 100).map({False : '< 100', True : '> 100'})
mask

In [ ]:
df.groupby(mask)['Valeur_fonciere'].mean().astype(int)

In [ ]:
#rappel base python = des listes partout
df.groupby(['Region', df.Code_departement , mask])['Valeur_fonciere'].mean().astype(int).to_frame().head()

In [ ]:
df.groupby(['Region'])['Valeur_fonciere'].apply(lambda x : 'good' if np.mean(x)< 300000 else 'bad')

# Reshaping

* Il existe différentes méthodes pour effectuer un reshape d’un dataframe (ex inverser ligne colonne). Rappelons que pandas comme python permet d’aboutir au même résultat avec des fonctions différentes. 
* L’objectif de cette section n’est pas d’être exhaustif mais plutôt de montrer quelle sont les bonnes pratiques en la matière et les pièges à éviter. Pour cela nous allons nous servir des similitudes entre groupby et pivot. 
* On conseil au formé de suivre ce lien au besoin qui illustre bien ces mécanismes : https://pandas.pydata.org/docs/user_guide/reshaping.html
* Dans cette section nous utiliserons une colonne catégorielle des surfaces avec la fonction cut, on commence par l’ajouter à df. En effet il est plus judicieux de comparer des surfaces de même ordre de grandeur


In [ ]:
df = pd.read_pickle('df')
df.shape
df.head(2)

df.Valeur_fonciere = df.Valeur_fonciere.fillna(0).astype(int)

## Categories

In [ ]:
pd.cut(df.Surface_reelle_bati, bins= [0,10,100,10000,100000]).value_counts().sort_index()

In [ ]:
a = pd.cut(df.Surface_reelle_bati, bins= [0,10,100,1000])
df.groupby([df.code_region, a]).size().head()

In [ ]:
df['Surface_Categorie'] = pd.cut(df.Surface_reelle_bati.fillna(0), bins= [-1,0,10,100,10000,100000])

In [ ]:
df.groupby(['Region', df.Surface_Categorie]).size()

## pivot

Il est très important lorsque on applique un reshape sur un dataframe de vérifier que les colonnes utilisées respectent une forme d’unicité des valeurs. 
1. Ce point est illustré avec la fonction pivot qui est la plus condensée pour du reshaping. Si on essaye de faire un pivot sur df region et categorie de surface cela génère une erreur : Index contains duplicate entries, cannot reshape.
1. Un moyen simple pour reshape ce type de dataframe et unifier les valeurs, est d’utiliser un groupby. Notons que nous pouvons arriver au même résultat en utilisant la fonction avancée de pivot = pivot table avec une fonction d’agrégation. La fonction stack sera expliqué dans la section suivante. 

In [ ]:
df.pivot(index = 'Region',columns= 'Surface_Categorie' , values= 'Valeur_fonciere')

In [ ]:
df.groupby(['Region', df.Surface_Categorie]).Valeur_fonciere.mean().fillna(0).astype(int)

In [ ]:
df.pivot_table(index = 'Region', columns = df.Surface_Categorie, values='Valeur_fonciere', aggfunc=np.mean).stack()

## multi index & stack

Dans la suite nous travaillons avec le dataframe groupé df2 (mean des valeurs foncières). Il est important de noter que df2 possède un multi index ce que nous pouvons vérifier avec df2.index. Le multi index est composé de level d’index.
1. Pour retrouver un df avec un unique level d’index il faut utiliser la commande reset_index, l’opération inverse se fait avec  set_index (colonnes) .
1. La caractéristique de pivot table et de basculer un level d’index en colonne cela peut être réalisé sur le dataframe multi-index df2  avec la commande unstack (En argument nous pouvons décider quel level d’index sera passé en colonne de la même façon que l’argument columns dans pivot table). L’opération inverse se fait avec stack
1. En conclusion groupby + unstack est équivalent à pivot table. L’intérêt en mode notebook de travailler avec ce type de shape est qu’il est facile de faire un graphique comme on peut le voir dans cet exemple ou on calcule le pourcentage de valeur foncière. 
1. A titre d’exemple nous réalisons la même opération avec le dénombrement des habitations par surface et par région.


In [ ]:
df2 = df.groupby(['Region', df.Surface_Categorie]).Valeur_fonciere.mean().fillna(0).astype(int)
df2

In [ ]:
df2.index[:10]

In [ ]:
df2.reset_index().Surface_Categorie

In [ ]:
df2.reset_index().set_index(['Region', 'Surface_Categorie']).Valeur_fonciere

In [ ]:
df2.unstack()

In [ ]:
df2.unstack().stack()

In [ ]:
df.groupby(['Region', df.Surface_Categorie]).Valeur_fonciere.mean().fillna(0).astype(int)

In [ ]:
df.groupby(['Region', df.Surface_Categorie]).Valeur_fonciere.mean().fillna(0).astype(int).unstack().plot.barh(stacked = True)

In [ ]:
df2 = df.pivot_table(index = 'Region', columns = df.Surface_Categorie, values='Valeur_fonciere', aggfunc=np.mean)
df2
(100*df2.div(df2.sum(1), 0)).plot.barh(stacked = True)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

In [ ]:
df.groupby(['Region', df.Surface_Categorie]).size().unstack().plot.barh(stacked = True)

In [ ]:
df2 = df.groupby(['Region', df.Surface_Categorie]).size().unstack()
(100*df2.div(df2.sum(1), 0)).plot.barh(stacked = True, cmap = 'autumn')
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# TimeSerie

Les times series sont des particularités dans le traitement de données qui nécessite des fonctions spécifiques. On retrouve comme première étape celle de s’assurer que les données sont bien au format de donnée date-time (si besoin on utilise la fonction de pandas to_datetime vue plus avant).
1. Les formats datetime sont manipulables avec pandas par l’intermédiaire de la fonction dt qui permet  par exemple de sélectionner un range de date spécifique ici month. 
1. La colonne date est traité vis-à-vis de pandas de la même façon que toute autre type de colonne  ex value count et groupby. 
1. Il est souvent utile de travailler sur des range de date sur mesure la moyenne roulante est déjà bien connu dans le milieu bancaire.


In [ ]:
df.head(2)

## to Date time 

In [ ]:
df.Date_mutation.value_counts().sort_index()

In [ ]:
df['Date_mutation'] = pd.to_datetime(df['Date_mutation'])

## Manip dt

In [ ]:
df.Date_mutation.dt.month.value_counts().sort_index()

In [ ]:
grouped = df.groupby(df.Date_mutation.dt.month)
grouped.Valeur_fonciere.sum().plot()

In [ ]:
grouped = df.groupby([df.Region, df.Date_mutation.dt.month])
grouped.Valeur_fonciere.sum().unstack(0)

In [ ]:
grouped = df.groupby([df.Region, df.Date_mutation.dt.month])
grouped.Valeur_fonciere.sum().unstack(0).cumsum().plot(figsize = (20,5) )

# rolling 

In [ ]:
dfparis = df[df.Region == 'Île-de-France']

In [ ]:
dfparis.set_index('Date_mutation').Valeur_fonciere.plot()

In [ ]:
a = dfparis.set_index('Date_mutation').sort_index()
a.Valeur_fonciere.rolling('7d').sum().plot()